In [3]:
!pip install googletrans deep-translator

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 33.6 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 3.5 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-3.0.0-py3-none-any.whl size=15718 sha256=8fb46a8a40d95a868cd067e1d971786866537991da8dd64640cf8b0215f5cbab
  Stored in directory: /root/.cache/pip/wheels/b3/81/ea/8b030407f8ebfc2f857814e086bb22ca2d4fea1a7be63652ab
Successfully built googletrans
  Attempting uninstall: h11
   

In [4]:
import numpy as np
import pandas as pd

df = pd.read_csv('/kaggle/input/dlsprint3/test.csv')
df.head()

,ID,Problem
0,0,একটি কেক-কে সরলরৈখিকভাবে 2 বার কেটে সর্বোচ্চ ক...
1,1,একটি পুকুরের উপর 100 টি পাথর রাখা আছে। প্রথমে ...
2,2,"ধরো $f(x) = x^{67-x^{67-x^{67-\dots}}}$, যেখান..."
3,3,সামিন ও স্বর্গ গণনার জন্য শুধু 0 আর 1 ব্যবহার ...
4,4,"A, B, C তিনটি স্ট্যান্ড। A স্ট্যান্ড থেকে 2 টি..."


In [5]:
import re
import pandas as pd
from deep_translator import GoogleTranslator

def translate_bangla_to_english(text):
    try:
        translator = GoogleTranslator(source='bn', target='en')
        translated_text = translator.translate(text)
        if translated_text is None:
            translated_text = ""
        return translated_text
    except Exception as e:
        print(f"Translation error for text: {text}\nError: {e}")
        return text

def process_text_with_latex(input_text):
    latex_pattern = r'\$.*?\$'
    latex_expressions = re.findall(latex_pattern, input_text)
    bangla_parts = re.split(latex_pattern, input_text)
    translated_parts = [translate_bangla_to_english(part) for part in bangla_parts]
    final_text = ''

    for i in range(len(translated_parts)):
        final_text += translated_parts[i] if translated_parts[i] is not None else ""
        if i < len(latex_expressions):
            final_text += latex_expressions[i]
            
    return final_text
    
translated_problems = []

for index, row in df.iterrows():
    bangla_text = row['Problem']
    translated_text = process_text_with_latex(bangla_text)  # Translate the problem text
    translated_problems.append({
        'ID': row['ID'],
        'Problem': translated_text
    })


translated_df = pd.DataFrame(translated_problems)

In [6]:
translated_df.head()

,ID,Problem
0,0,What is the maximum number of parts a cake can...
1,1,100 stones are placed on a pond. First a frog ...
2,2,"hold on$f(x) = x^{67-x^{67-x^{67-\dots}}}$, wh..."
3,3,Samin and Swarga use only 0 and 1 for calculat...
4,4,"A, B, C are three stands. It takes 3 moves to ..."


In [7]:
translated_df['Problem'][2]

'hold on$f(x) = x^{67-x^{67-x^{67-\\dots}}}$, where$x \\neq 0$, if$f(n) = 64$Yes, then$n^n$How many quotients will be left when divided by 11?'

In [ ]:
import re
import torch
from transformers import pipeline

pipe = pipeline("text-generation", model="AI-MO/NuminaMath-7B-TIR", torch_dtype=torch.bfloat16, device_map="auto")

messages = [
    {"role": "user", "content": "hold on$f(x) = x^{67-x^{67-x^{67-\\dots}}}$, where$x \\neq 0$, if$f(n) = 64$Yes, then$n^n$How many quotients will be left when divided by 11?"},
]
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

gen_config = {
    "max_new_tokens": 1024,
    "do_sample": True,
    "stop_strings": ["```output"],
    "temperature": 0.5,  # Control randomness
    "top_k": 50,  # Limit to top 50 tokens
    "top_p": 0.85,  # Nucleus sampling
    "repetition_penalty": 1.2,  # Penalty for repeating tokens
    "length_penalty": 1.0,  # Length penalty
    "num_return_sequences": 3,  # Return 3 different sequences
    "pad_token_id": pipe.tokenizer.eos_token_id,  # Use eos token for padding
    "eos_token_id": pipe.tokenizer.eos_token_id,  # Define end of sequence
    "tokenizer": pipe.tokenizer,
}

outputs = pipe(prompt, **gen_config)
print(outputs)
text = outputs[0]["generated_text"]
# print(text)

# WARNING: This code will execute the python code in the string. We show this for eductional purposes only.
# Please refer to our full pipeline for a safer way to execute code.
python_code = re.findall(r"```python(.*?)```", text, re.DOTALL)[0]
print(python_code)
exec(python_code)


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [10]:
import signal
import tempfile
import os
import subprocess
import re
from contextlib import contextmanager

class PythonREPL:
    def __init__(self, timeout=5):
        self.timeout = timeout

    @contextmanager
    def time_limit(self, seconds):
        def signal_handler(*_):
            raise TimeoutError(f"Timed out after {seconds} seconds.")

        signal.signal(signal.SIGALRM, signal_handler)
        signal.alarm(seconds)
        try:
            yield
        finally:
            signal.alarm(0)

    def __call__(self, query):
        query = "import math\nimport numpy as np\nimport sympy as sp\n" + query
        query = query.strip().split("\n")
        if "print(" not in query[-1]:
            if "#" in query[-1]:
                query[-1] = query[-1].split("#")[0]
            query[-1] = "print(" + query[-1] + ")"
        query = "\n".join(query)
        with tempfile.TemporaryDirectory() as temp_dir:
            temp_file_path = os.path.join(temp_dir, "tmp.py")
            with open(temp_file_path, "w", encoding="utf-8") as f:
                f.write(query)
            with self.time_limit(self.timeout):
                result = subprocess.run(
                    ["python3", temp_file_path],
                    capture_output=True,
                    check=False,
                    text=True,
                    timeout=self.timeout,
                )
                if result.returncode == 0:
                    output = result.stdout
                    return True, output.strip()
                error_msg = result.stderr.strip()
                msgs = error_msg.split("\n")
                new_msgs = []
                want_next = False
                for m in msgs:
                    if "Traceback" in m:
                        new_msgs.append(m)
                    elif m == msgs[-1]:
                        new_msgs.append(m)
                    elif temp_file_path in m:
                        st = m.index('"/') + 1 if '"/' in m else 0
                        ed = m.index(temp_file_path) + 1 if temp_file_path in m else None
                        clr = m[st:ed] if not ed else m[st:]
                        m = m.replace(clr, "")
                        new_msgs.append(m)
                        want_next = True
                    elif want_next:
                        new_msgs.append(m)
                        want_next = False
                error_msg = "\n".join(new_msgs)
                return False, error_msg.strip()
            

def execute_completion(executor, completion, return_status, last_code_block):
    executions = re.findall(r"```python(.*?)```", completion, re.DOTALL)
    if len(executions) == 0:
        return completion, False if return_status else completion
    if last_code_block:
        executions = [executions[-1]]
    outputs = []
    successes = []
    for code in executions:
        success = False
        for lib in ("subprocess", "venv"):
            if lib in code:
                output = f"{lib} is not allowed"
                outputs.append(output)
                successes.append(success)
                continue
        try:
            success, output = executor(code)
        except TimeoutError as e:
            print("Code timed out")
            output = e
        if not success and not return_status:
            output = ""
        outputs.append(output)
        successes.append(success)
    output = str(outputs[-1]).strip()
    success = successes[-1]
    if return_status:
        return output, success
    return output


def postprocess_completion(text, return_status, last_code_block):
    executor = PythonREPL()
    result = execute_completion(executor, text, return_status=return_status, last_code_block=last_code_block)
    del executor
    return result

In [21]:
import math

def process_number(value):
    try:
        # Attempt to convert the string to a float
        num = float(value)
        
        if math.isinf(num):
            return float('inf') if num > 0 else float('-inf')
        
        # Check if it's already a non-negative integer
        if num.is_integer() and num >= 0:
            return int(num)
        
        # Handle float values as per your custom rounding rule
        elif isinstance(num, float):
            if num >= 0:
                return math.floor(num) if num < math.floor(num) + 0.5 else math.ceil(num)
            else:
                return abs(math.floor(num))  # Convert negative to positive and round down
                
    except ValueError:
        # If the string cannot be converted to a number, return None
        return None

In [15]:
# pipe = pipeline("text-generation", model="AI-MO/NuminaMath-7B-TIR", torch_dtype=torch.bfloat16, device_map="auto")


for index, problem in translated_df.iterrows():
    messages = [
        {"role": "user", "content": problem['Problem']}
    ]

    predictions = None
    prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

    gen_config = {
        "max_new_tokens": 1024,
        "do_sample": False,
        "stop_strings": ["```output"],  # Generate until Python code block is complete
        "tokenizer": pipe.tokenizer,
    }

    outputs = pipe(prompt, **gen_config)
    text = outputs[0]["generated_text"]

    code_result, success = postprocess_completion(text, return_status=True, last_code_block=True)
    if success:
        prediction = code_result
    print(f'index:{index}', prediction)
    clean_answer = process_number(prediction)
    translated_df.at[index, 'Answer'] = clean_answer
    print(clean_answer)
    
# result_df = pd.DataFrame(translated_problems)


index:0 4
4
index:1 10
10
index:2 (3.9999999999999996, 255.99999999999972, 2.999999999999716)
None
index:3 225
225
index:4 100
100
index:5 100
100
index:6 17
17
index:7 [(1, 11, 21), (1, 11, 24), (1, 11, 27), (1, 11, 30), (1, 12, 23), (1, 12, 26), (1, 12, 29), (1, 13, 22), (1, 13, 25), (1, 13, 28), (1, 14, 21), (1, 14, 24), (1, 14, 27), (1, 14, 30), (1, 15, 23), (1, 15, 26), (1, 15, 29), (1, 16, 22), (1, 16, 25), (1, 16, 28), (1, 17, 21), (1, 17, 24), (1, 17, 27), (1, 17, 30), (1, 18, 23), (1, 18, 26), (1, 18, 29), (1, 19, 22), (1, 19, 25), (1, 19, 28), (1, 20, 21), (1, 20, 24), (1, 20, 27), (1, 20, 30), (2, 11, 23), (2, 11, 26), (2, 11, 29), (2, 12, 22), (2, 12, 25), (2, 12, 28), (2, 13, 21), (2, 13, 24), (2, 13, 27), (2, 13, 30), (2, 14, 23), (2, 14, 26), (2, 14, 29), (2, 15, 22), (2, 15, 25), (2, 15, 28), (2, 16, 21), (2, 16, 24), (2, 16, 27), (2, 16, 30), (2, 17, 23), (2, 17, 26), (2, 17, 29), (2, 18, 22), (2, 18, 25), (2, 18, 28), (2, 19, 21), (2, 19, 24), (2, 19, 27), (2, 19, 30)

OverflowError: cannot convert float infinity to integer

In [20]:
translated_df['Answer'][70]

16

In [22]:
# # pipe = pipeline("text-generation", model="AI-MO/NuminaMath-7B-TIR", torch_dtype=torch.bfloat16, device_map="auto")

# start_index = 71
# for index, problem in translated_df.iterrows():
#     if index < start_index:
#         continue
        
#     messages = [
#         {"role": "user", "content": problem['Problem']}
#     ]

#     predictions = None
#     prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

#     gen_config = {
#         "max_new_tokens": 1024,
#         "do_sample": False,
#         "stop_strings": ["```output"],  # Generate until Python code block is complete
#         "tokenizer": pipe.tokenizer,
#     }

#     outputs = pipe(prompt, **gen_config)
#     text = outputs[0]["generated_text"]

#     code_result, success = postprocess_completion(text, return_status=True, last_code_block=True)
#     if success:
#         prediction = code_result
#     print(f'index:{index}', prediction)
#     clean_answer = process_number(prediction)
#     translated_df.at[index, 'Answer'] = clean_answer
#     print(clean_answer)
    
# # result_df = pd.DataFrame(translated_problems)


index:71 inf
inf
index:72 16
16
index:73 False
None
index:74 16
16
index:75 17
17
index:76 96.0000000000000
96
index:77 6
6
index:78 57.0
57
index:79 62281
62281
index:80 (137, [1, 36, 100])
None
index:81 0
0
index:82 7
7
index:83 43
43
index:84 24
24
index:85 6745674
6745674
index:86 6745674
6745674
index:87 0
0
index:88 6
6
index:89 ((1, 3, 5, 7), 105)
None
index:90 28
28
index:91 169
169
index:92 8
8
index:93 24.0
24
index:94 1012
1012
index:95 {b: 2, r: 1}
None
index:96 (80960, [2068, 2288, 2508, 2728, 2948, 6160, 6380, 6600, 6820, 8052, 8272, 8492, 8712, 8932])
None
index:97 (80960, [2068, 2288, 2508, 2728, 2948, 6160, 6380, 6600, 6820, 8052, 8272, 8492, 8712, 8932])
None
index:98 [71]
None
index:99 7.5
8


In [23]:
translated_df

,ID,Problem,Answer
0,0,What is the maximum number of parts a cake can...,4
1,1,100 stones are placed on a pond. First a frog ...,10
2,2,"hold on$f(x) = x^{67-x^{67-x^{67-\dots}}}$, wh...",None
3,3,Samin and Swarga use only 0 and 1 for calculat...,225
4,4,"A, B, C are three stands. It takes 3 moves to ...",100
...,...,...,...
95,95,A box contains some red and blue balls. If one...,None
96,96,A four-digit integer is divisible by 11. If th...,None
97,97,How many positive integers are there such that...,None
98,98,"If the sum of 3 consecutive integers is 216, w...",None


In [24]:
temp_df = translated_df

In [29]:
temp_df['Answer'] = temp_df['Answer'].replace([np.inf, -np.inf], 0)
temp_df['Answer'] = temp_df['Answer'].where(temp_df['Answer'].notnull() & (temp_df['Answer'] >= 0), 0).astype(int)

/tmp/ipykernel_29/4125251366.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  temp_df['Answer'] = temp_df['Answer'].replace([np.inf, -np.inf], 0)


In [30]:
temp_df

,ID,Problem,Answer
0,0,What is the maximum number of parts a cake can...,4
1,1,100 stones are placed on a pond. First a frog ...,10
2,2,"hold on$f(x) = x^{67-x^{67-x^{67-\dots}}}$, wh...",0
3,3,Samin and Swarga use only 0 and 1 for calculat...,225
4,4,"A, B, C are three stands. It takes 3 moves to ...",100
...,...,...,...
95,95,A box contains some red and blue balls. If one...,0
96,96,A four-digit integer is divisible by 11. If th...,0
97,97,How many positive integers are there such that...,0
98,98,"If the sum of 3 consecutive integers is 216, w...",0


In [31]:
final_df = temp_df.drop(columns=['Problem'])
final_df.to_csv('submission.csv', index=False)

In [34]:
from IPython.display import FileLink
FileLink('submission.csv')

/kaggle/working/submission.csv

In [2]:
model_config = pipe.model.config
print(model_config)

LlamaConfig {
  "_name_or_path": "AI-MO/NuminaMath-7B-TIR",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 100000,
  "eos_token_id": 100001,
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 11008,
  "max_position_embeddings": 4096,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 30,
  "num_key_value_heads": 32,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 10000.0,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.45.1",
  "use_cache": true,
  "vocab_size": 102400
}



In [34]:
# messages = [
#     {"role": "user", "content": "What is the maximum number of pieces a cake can be divided into by cutting it 2 times in a straight line?"}
# ]

# # Generate answers multiple times (e.g., 3 times)
# predictions = None
# prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

# gen_config = {
#     "max_new_tokens": 1024,
#     "do_sample": False,
#     "stop_strings": ["```output"],  # Generate until Python code block is complete
#     "tokenizer": pipe.tokenizer,
# }

# outputs = pipe(prompt, **gen_config)
# text = outputs[0]["generated_text"]

# code_result, success = postprocess_completion(text, return_status=True, last_code_block=True)
# if success:
#     prediction = code_result
# # Extract the Python code from the output
# # python_code = re.findall(r"```python(.*?)```", text, re.DOTALL)
# # if python_code:  # If there's Python code, attempt to execute it
# #     executor = PythonREPL()
    
# #     # Execute the Python code
# #     result_output, success = execute_completion(executor, python_code[0], return_status=True, last_code_block=True)
    
# #     if success:
# #         # Since the code contains a print statement, capture the last printed output
# #         # This assumes the last printed output is the desired result
# #         predictions.append(result_output)
# #         print(result_output)
# #     else:
# #         predictions.append(None)
# # else:
# #     predictions.append(None)  # Append None if no code was found

# # Get the majority vote from the predictions
# # print("Gdfgsg")

4
